In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pathlib
import random
import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
from tensorflow.keras import layers
import PIL

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

images_path = os.path.join(os.path.realpath('..'), "input", "kermany2018", "OCT2017 ")
images_path = pathlib.Path(images_path)

image_count = len(list(images_path.glob('*/*.jpeg')))
print(image_count)


batch_size = 32
img_height = 180
img_width = 180

0


In [2]:
test_ds = image_dataset_from_directory(
os.path.join(images_path, "test"),
  validation_split=0,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = image_dataset_from_directory(
os.path.join(images_path, "val"),
  validation_split=0,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

train_ds = image_dataset_from_directory(
  os.path.join(images_path, "train"),
  validation_split=0,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

num_classes = 4


Found 968 files belonging to 4 classes.
Found 32 files belonging to 4 classes.
Found 83484 files belonging to 4 classes.


In [3]:
model = tf.keras.Sequential([
  layers.Flatten(),
  layers.Dense(num_classes),
  layers.Dense(1),
  layers.Dense(128, activation='relu'),
  layers.Dense(256, activation='relu'),
  layers.Dense(512, activation='relu'),
  layers.Dense(1024, activation='relu'),
  layers.Dense(512, activation='relu'),
  layers.Dense(256, activation='relu'),
  layers.Dense(128, activation='relu'),
])

In [4]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1
)

model.summary()

2609/2609 [==============================] - 372s 142ms/step - loss: 5.6593 - accuracy: 0.4412 - val_loss: 4.8520 - val_accuracy: 0.2500
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 97200)             0         
_________________________________________________________________
dense (Dense)                (None, 4)                 388804    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               256       
_________________________________________________________________
dense_3 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_4 (Dense)              (None, 512)           

In [5]:
model.evaluate(test_ds)

31/31 [==============================] - 3s 87ms/step - loss: 4.8520 - accuracy: 0.2500


[4.852030277252197, 0.25]

In [ ]:
# Calculate F1 score
from sklearn import metrics
# metrics.f1_score(y_actual, y_pred, labels=[0, 1, 2], average='macro')
y_pred =[]
y_true = []

i = 0
for batch, lbls in test_ds.as_numpy_iterator():
    prediction = model.predict_on_batch(batch)
    
    for i in range(len(prediction)):
        y_true.append(lbls[i])
        prediction_single = np.argmax(prediction[i])
        y_pred.append(prediction_single)
#     print(batch.shape)
#     print(lbls.shape)
print(y_true)
print(y_pred)
    
print(metrics.f1_score(y_true, y_pred, labels=[0, 1, 2, 3], average='macro'))